In [7]:
from selenium import webdriver
import time
import ssl
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
ssl._create_default_https_context = ssl._create_unverified_context
driver = webdriver.Chrome(executable_path=r"C:\Users\kashi\go\chromedriver.exe")
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import numpy as np
from tensorflow.keras.models import load_model
import cv2
import os, shutil
from glob import glob
import math
import tqdm
import tensorflowjs as tfjs
import numpy
import pandas
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score
import sys
from tensorflow.keras.applications import EfficientNetB0
import efficientnet.keras as efn
import re
import youtube_dl

print("Awaiting Video Player")
model=load_model(r'C:\Users\kashi\Downloads\proj\NPDIv3fullofull.12-0.17.h5')
saved_url=""
wait=WebDriverWait(driver, 500)

def url_changed(saved_url):
    wait.until(
        lambda driver: driver.current_url!=saved_url)
    return True;

def predict():
    prediction_images = []
    ele=WebDriverWait(driver,500).until(EC.presence_of_element_located((By.CLASS_NAME, "ytd-watch-flexy") ))
    print("Video Player found")
    start = time.time()
    #element.screenshot(r"C:\Users\kashi\Downloads\proj\temp\shot"+str(x)+".png") 
    video_url = driver.current_url
    ydl_opts = {}
    ydl = youtube_dl.YoutubeDL(ydl_opts)
    info_dict = ydl.extract_info(video_url, download=False)
    formats = info_dict.get('formats',None)
    print("Obtaining frames")
    for f in formats:
        if f.get('format_note',None) == '144p':
            url = f.get('url',None)
            cap = cv2.VideoCapture(url)
            if not cap.isOpened():
                print('video not opened')
                exit(-1)
            x=0
            count=0
            while x<10:
                ret, frame = cap.read()
                if not ret:
                    break
                filename =r"C:\Users\kashi\Downloads\proj\temp\shot"+str(x)+".png"
                x+=1
                cv2.imwrite(filename.format(count), frame)
                count+=100
                cap.set(1,count)
                if cv2.waitKey(30)&0xFF == ord('q'):
                    break
            cap.release()
            
    print(str(x)+" frames obtained successfully")
    print("Running Prediction Model")

    images = glob(r"C:\Users\kashi\Downloads\proj\temp\*.png")

    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = numpy.array(img)
        img = img/255
        prediction_images.append(img)

    prediction_images=numpy.array(prediction_images)
    pred=model.predict(prediction_images)
    y_pred = numpy.argmax(pred, axis=-1)

    for i in range(0,len(y_pred)):
        if y_pred[i]==1 and pred[i][1]<=0.95:
            y_pred[i]=0
    print(numpy.sum(y_pred),math.ceil(10*len(prediction_images)/100))
    confiporn=[]
    confinonporn=[]
    for i in range(0,len(y_pred)):
        if y_pred[i]==1:
            confiporn.append(pred[i][1])
        else:
            confinonporn.append(pred[i][0])
    confiporn=numpy.array(confiporn)
    confinonporn=numpy.array(confinonporn)

    if(numpy.sum(y_pred)>=math.ceil(10*len(prediction_images)/100)):
        prediction=1
        confidence=numpy.mean(confiporn)
    else:
        prediction=0
        confidence=numpy.mean(confinonporn)
    confi = numpy.round(confidence*100)

    end=time.time()
    print("Predicted to be of type "+str(prediction)+" in "+str(round(end-start,3))+" seconds") 

    if prediction==0:
        driver.execute_script("document.getElementsByClassName('html5-video-container')[0].style.filter='blur(100px)';")

    print("...for "+driver.title)
    folder=r"C:\Users\kashi\Downloads\proj\temp"
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

    return driver.current_url

saved_url=predict()

while True:
    if url_changed(saved_url):
        print("Abandoned Current Video Player")
        saved_url=predict()    

Awaiting Video Player
Video Player found
[youtube] azG9nwlpMRA: Downloading webpage
Obtaining frames
10 frames obtained successfully
Running Prediction Model
0 1
Predicted to be of type 0 in 8.36 seconds
...for Tom Walker - Just You and I (Giulia and Marcello Fan Video) - YouTube


WebDriverException: Message: chrome not reachable
  (Session info: chrome=89.0.4389.90)
